In [325]:
#Read Input & Assign Variables
import csv
import math
with open('tc1.csv', newline='') as input:
    data = tuple(csv.reader(input))

M = int(data[0][0])
N = int(data[0][1])
K = int(data[1][0])
alpha = int(data[2][0])
print(M)
print(N)

init_speed = ()
data_size = ()
factor = ()

for i in range (3,3+K):
    init_speed = (*init_speed, int(data[i][1]))
    data_size = (*data_size, int(data[i][2]))
    factor = (*factor, int(data[i][3]))
total_data = sum(list(data_size))
data_remaining = list(data_size)

FileNotFoundError: [Errno 2] No such file or directory: 'tc1.csv'

In [ ]:
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed_data_map = []
for i in range (0,len(speed_map)):
    speed_data_map = (*speed_data_map, int(speed_map[i][1]))


In [ ]:
def size_index(data):
    index_map = []
    for i in range (0,K):
        max_index = data.index(max(data))
        index_map.append(max_index)
        data[max_index] = -1
    return index_map

In [ ]:
def target_factor(index_map,users,flag):
    sent_data = 0
    collocated_factor_list = [0]*M
    for i in range (0,users):
        collocated_factor_list[i] = factor[index_map[i]-1]/100
    
    for i in range (0,users):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))
        speed = init_speed[index_map[i]-1] * (1-collocated_factor)
        speed = math.floor(speed + 10e-6)
        sent_data = sent_data + speed_data_map[speed]
        if flag == 1:
            data_remaining[index_map[i]-1] = data_remaining[index_map[i]-1] - speed_data_map[speed]
            if (data_remaining[index_map[i]-1] < 0):
                data_remaining[index_map[i]-1] = 0

    target_current = (math.floor((sent_data/total_data)*100))
    return target_current

In [ ]:
target_ideal = 100/N
user_index_map_d = [0]*M
user_index_map_q = [0]*M
print(data_remaining)
for i in range (0,N):       #Columns
    
    data_temp = data_remaining.copy()
    size_index_map = size_index(data_temp)         #shows the index of largest and the subsequent largest data_size 
    margin = 100
    for j in range (0,M):
        user_index_map_d[j] = size_index_map[j]+1       
        target_curr = target_factor(user_index_map_d,j+1,0)
        if abs(target_curr-target_ideal) > margin:
            target_factor(user_index_map_q,j,1)
            break
        else:
            user_index_map_q = user_index_map_d.copy()
            margin = abs(target_curr-target_ideal)


print(data_remaining)
            

        

[26000, 26244, 1504, 4585, 252, 525, 2328, 1630, 2562, 741, 1108, 1310, 2345, 1705, 540, 450, 2695, 213, 2026, 2567, 1937, 1656, 253, 1680, 1963, 2275, 1406, 2141, 3620, 540, 1317, 4074, 994, 2140, 1630, 212, 640, 1347, 259, 442, 1560, 229, 542, 652, 2113, 352, 2471, 254, 956, 2447, 457, 3104, 1920, 2514, 245, 2114, 1573, 1016, 2740, 3240, 1197, 1133, 418, 2927, 222, 3445, 459, 212, 945, 2179, 2252, 3857, 245, 1506, 2020, 1550, 1040, 542, 1827, 1582]
[1114, 1358, 1504, 0, 252, 525, 0, 1630, 0, 741, 1108, 1310, 0, 1705, 540, 450, 0, 213, 0, 0, 0, 1656, 253, 1680, 0, 0, 1406, 0, 0, 540, 1317, 0, 994, 251, 1630, 212, 640, 1347, 259, 442, 1560, 229, 542, 652, 0, 352, 0, 254, 956, 0, 457, 0, 0, 0, 245, 0, 1573, 1016, 0, 0, 1197, 1133, 418, 0, 222, 0, 459, 212, 945, 0, 0, 0, 245, 1506, 0, 1550, 1040, 542, 0, 1582]
